In [1]:
# 이 주피터 노트북의 코드는 nltk에서 구현한 bleu 스코어 소스코드입니다.
# reference: https://www.nltk.org/_modules/nltk/translate/bleu_score.html

In [2]:
import math
import warnings
import sys
from fractions import Fraction
from nltk import ngrams
from collections import Counter

In [3]:
r1 = 'you know I always love you so much'.split(' ')
r2 = 'know right I always love you a lot'.split(' ')
h1 = 'know that I always love you that much'.split(' ')
h2 = 'you love I always'.split(' ')
h3 = 'I I I I I'.split(' ')

In [4]:
###################

In [5]:
class SmoothingFunction:

    def __init__(self, epsilon=0.1, alpha=5, k=5):
        self.epsilon = epsilon
        self.alpha = alpha
        self.k = k


    def method0(self, p_n, *args, **kwargs):
        """
        No smoothing.
        """
        p_n_new = []
        for i, p_i in enumerate(p_n):
            if p_i.numerator != 0:
                p_n_new.append(p_i)
            else:
                _msg = str(
                    "\nThe hypothesis contains 0 counts of {}-gram overlaps.\n"
                    "Therefore the BLEU score evaluates to 0, independently of\n"
                    "how many N-gram overlaps of lower order it contains.\n"
                    "Consider using lower n-gram order or use "
                    "SmoothingFunction()"
                ).format(i + 1)
                warnings.warn(_msg)
                # When numerator==0 where denonminator==0 or !=0, the result
                # for the precision score should be equal to 0 or undefined.
                # Due to BLEU geometric mean computation in logarithm space,
                # we we need to take the return sys.float_info.min such that
                # math.log(sys.float_info.min) returns a 0 precision score.
                p_n_new.append(1/100000000)
        return p_n_new


    def method1(self, p_n, *args, **kwargs):
        return [
            (p_i.numerator + self.epsilon) / p_i.denominator
            if p_i.numerator == 0
            else p_i
            for p_i in p_n
        ]


    def method2(self, p_n, *args, **kwargs):
        return [
            Fraction(p_n[i].numerator + 1, p_n[i].denominator + 1, _normalize=False)
            if i != 0
            else p_n[0]
            for i in range(len(p_n))
        ]


    def method3(self, p_n, *args, **kwargs):
        incvnt = 1  # From the mteval-v13a.pl, it's referred to as k.
        for i, p_i in enumerate(p_n):
            if p_i.numerator == 0:
                p_n[i] = 1 / (2 ** incvnt * p_i.denominator)
                incvnt += 1
        return p_n


    def method4(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        incvnt = 1
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        for i, p_i in enumerate(p_n):
            if p_i.numerator == 0 and hyp_len > 1:
                # incvnt = i + 1 * self.k / math.log(
                #     hyp_len
                # )  # Note that this K is different from the K from NIST.
                # p_n[i] = incvnt / p_i.denominator\
                numerator = 1 / (2 ** incvnt * self.k / math.log(hyp_len))
                p_n[i] = numerator / p_i.denominator
                incvnt += 1
        return p_n


    def method5(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        m = {}
        # Requires an precision value for an addition ngram order.
        p_n_plus1 = p_n + [modified_precision(references, hypothesis, 5)]
        m[-1] = p_n[0] + 1
        for i, p_i in enumerate(p_n):
            p_n[i] = (m[i - 1] + p_i + p_n_plus1[i + 1]) / 3
            m[i] = p_n[i]
        return p_n


    def method6(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        # This smoothing only works when p_1 and p_2 is non-zero.
        # Raise an error with an appropriate message when the input is too short
        # to use this smoothing technique.
        assert p_n[2], "This smoothing method requires non-zero precision for bigrams."
        for i, p_i in enumerate(p_n):
            if i in [0, 1]:  # Skips the first 2 orders of ngrams.
                continue
            else:
                pi0 = 0 if p_n[i - 2] == 0 else p_n[i - 1] ** 2 / p_n[i - 2]
                # No. of ngrams in translation that matches the reference.
                m = p_i.numerator
                # No. of ngrams in translation.
                l = sum(1 for _ in ngrams(hypothesis, i + 1))
                # Calculates the interpolated precision.
                p_n[i] = (m + self.alpha * pi0) / (l + self.alpha)
        return p_n


    def method7(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        p_n = self.method4(p_n, references, hypothesis, hyp_len)
        p_n = self.method5(p_n, references, hypothesis, hyp_len)
        return p_n


In [6]:
def brevity_penalty(closest_ref_len, hyp_len):
    '''
    closest_ref_len: 후보문장과 가장 길이 차이가 작은 문장의 길이
    hyp_len: 후보문장의 길이의 길이
    '''
    if hyp_len > closest_ref_len:
        return 1
    # If hypothesis is empty, brevity penalty = 0 should result in BLEU = 0.0
    elif hyp_len == 0:
        return 0
    else:
        return math.exp(1 - closest_ref_len / hyp_len)


In [7]:
def closest_ref_length(references, hyp_len):
    ref_lens = (len(reference) for reference in references)
    closest_ref_len = min(
        ref_lens, key=lambda ref_len: (abs(ref_len - hyp_len), ref_len)
    )
    return closest_ref_len

In [8]:
counts = Counter(ngrams(h1, 1))
max_counts = {}
for reference in [r1, r2]:
    reference_counts = (
        Counter(ngrams(reference, 1)) if len(reference) >= 1 else Counter()
    )
    for ngram in counts:
        max_counts[ngram] = max(max_counts.get(ngram, 0), reference_counts[ngram])

In [9]:
max_counts, counts

({('know',): 1,
  ('that',): 0,
  ('I',): 1,
  ('always',): 1,
  ('love',): 1,
  ('you',): 2,
  ('much',): 1},
 Counter({('know',): 1,
          ('that',): 2,
          ('I',): 1,
          ('always',): 1,
          ('love',): 1,
          ('you',): 1,
          ('much',): 1}))

In [10]:
def modified_precision(references, hypothesis, n):
    # Extracts all ngrams in hypothesis
    # Set an empty Counter if hypothesis is empty.
    counts = Counter(ngrams(hypothesis, n)) if len(hypothesis) >= n else Counter()
    # Extract a union of references' counts.
    # max_counts = reduce(or_, [Counter(ngrams(ref, n)) for ref in references])
    max_counts = {}
    
    # hypothesis의 ngram 단어들에 대해서 아래의 작업을 수행해서 ngram 마다 최대 카운트 수를 구한다.
    # -> ngram 단어가 reference에서 나온 카운트 수를 구함 (A)
    # -> 지금까지 나온 레퍼런스 중에서 ngram 단어가 나온 최대 카운트 수 (B)
    # -> A와 B 중 큰 수 구하기
    for reference in references:
        reference_counts = (
            Counter(ngrams(reference, n)) if len(reference) >= n else Counter()
        )
        for ngram in counts:
            max_counts[ngram] = max(max_counts.get(ngram, 0), reference_counts[ngram])
    print(f'At {n}-gram')
    print(f'max_counts: {max_counts}')
    print(f'counts: {counts}')

    # Assigns the intersection between hypothesis and references' counts.
    # print('Do clip = intersection')
        
    clipped_counts = {
        ngram: min(count, max_counts[ngram]) for ngram, count in counts.items()
    }
    print(f'clipped_counts: {clipped_counts}')

    numerator = sum(clipped_counts.values())
    # Ensures that denominator is minimum 1 to avoid ZeroDivisionError.
    # Usually this happens when the ngram order is > len(reference).
    denominator = max(1, sum(counts.values()))
    print(denominator)

    return Fraction(numerator, denominator, _normalize=False)

In [11]:
def corpus_bleu(
    list_of_references,
    hypotheses,
    weights=(0.25, 0.25, 0.25, 0.25),
    smoothing_function=None,
    auto_reweigh=False,
):
    p_numerators = Counter()  # Key = ngram order, and value = no. of ngram matches.
    p_denominators = Counter()  # Key = ngram order, and value = no. of ngram in ref.
    hyp_lengths, ref_lengths = 0, 0

    assert len(list_of_references) == len(hypotheses), (
        "The number of hypotheses and their reference(s) should be the " "same "
    )

    try:
        weights[0][0]
    except TypeError:
        weights = [weights]
    max_weight_length = max(len(weight) for weight in weights)

    # Iterate through each hypothesis and their corresponding references.
    for references, hypothesis in zip(list_of_references, hypotheses):
        print(references, hypotheses)
        # For each order of ngram, calculate the numerator and
        # denominator for the corpus-level modified precision.
        # 1 ~ N gram에 대해서 계산하기
        for i in range(1, max_weight_length + 1):
            # print(f'{i} -> r={references}   h={hypothesis}')
            p_i = modified_precision(references, hypothesis, i)
            p_numerators[i] += p_i.numerator
            p_denominators[i] += p_i.denominator
            print(f'p_{i} -> numerator={p_i.numerator} denominator={p_i.denominator} by modified_precision with {i}-Gram')

        # Calculate the hypothesis length and the closest reference length.
        # Adds them to the corpus-level hypothesis and reference counts.
        # brevity penalty를 구하기 위해서 계산함!!
        hyp_len = len(hypothesis)
        hyp_lengths += hyp_len
        ref_lengths += closest_ref_length(references, hyp_len)

    # Calculate corpus-level brevity penalty.
    bp = brevity_penalty(ref_lengths, hyp_lengths)
    print(f'brevity penalty: {bp}')

    print(f'p_numerator: {p_numerators}')
    print(f'p_denominators: {p_denominators}')
    # Collects the various precision values for the different ngram orders.
    p_n = [
        Fraction(p_numerators[i], p_denominators[i], _normalize=False)
        for i in range(1, max_weight_length + 1)
    ]

    # Returns 0 if there's no matching n-grams
    # We only need to check for p_numerators[1] == 0, since if there's
    # no unigrams, there won't be any higher order ngrams.
    if p_numerators[1] == 0:
        return 0 if len(weights) == 1 else [0] * len(weights)

    print(f'p_n: {p_n}')
    # If there's no smoothing, set use method0 from SmoothinFunction class.
    if not smoothing_function:
        smoothing_function = SmoothingFunction().method0
    # Smoothen the modified precision.
    # Note: smoothing_function() may convert values into floats;
    #       it tries to retain the Fraction object as much as the
    #       smoothing method allows.
    p_n = smoothing_function(
        p_n, references=references, hypothesis=hypothesis, hyp_len=hyp_lengths
    )
    print(f'p_n smoothing: {p_n}')

    bleu_scores = []
    for weight in weights:
        # Uniformly re-weighting based on maximum hypothesis lengths if largest
        # order of n-grams < 4 and weights is set at default.
        if auto_reweigh:
            if hyp_lengths < 4 and weight == (0.25, 0.25, 0.25, 0.25):
                weight = (1 / hyp_lengths,) * hyp_lengths

        s = (w_i * math.log(p_i) for w_i, p_i in zip(weight, p_n) if p_i > 0)
        s = bp * math.exp(math.fsum(s))
        bleu_scores.append(s)
    return bleu_scores[0] if len(weights) == 1 else bleu_scores

In [12]:
def sentence_bleu(
    references,
    hypothesis,
    weights=(0.25, 0.25, 0.25, 0.25),
    smoothing_function=None,
    auto_reweigh=False,
):
    return corpus_bleu(
        [references], [hypothesis], weights, smoothing_function, auto_reweigh
    )

In [13]:
r1

['you', 'know', 'I', 'always', 'love', 'you', 'so', 'much']

In [14]:
r2

['know', 'right', 'I', 'always', 'love', 'you', 'a', 'lot']

In [15]:
h1

['know', 'that', 'I', 'always', 'love', 'you', 'that', 'much']

In [16]:
h2

['you', 'love', 'I', 'always']

In [17]:
h3

['I', 'I', 'I', 'I', 'I']

In [18]:
sentence_bleu([r1, r2], h1)

[['you', 'know', 'I', 'always', 'love', 'you', 'so', 'much'], ['know', 'right', 'I', 'always', 'love', 'you', 'a', 'lot']] [['know', 'that', 'I', 'always', 'love', 'you', 'that', 'much']]
At 1-gram
max_counts: {('know',): 1, ('that',): 0, ('I',): 1, ('always',): 1, ('love',): 1, ('you',): 2, ('much',): 1}
counts: Counter({('that',): 2, ('know',): 1, ('I',): 1, ('always',): 1, ('love',): 1, ('you',): 1, ('much',): 1})
clipped_counts: {('know',): 1, ('that',): 0, ('I',): 1, ('always',): 1, ('love',): 1, ('you',): 1, ('much',): 1}
8
p_1 -> numerator=6 denominator=8 by modified_precision with 1-Gram
At 2-gram
max_counts: {('know', 'that'): 0, ('that', 'I'): 0, ('I', 'always'): 1, ('always', 'love'): 1, ('love', 'you'): 1, ('you', 'that'): 0, ('that', 'much'): 0}
counts: Counter({('know', 'that'): 1, ('that', 'I'): 1, ('I', 'always'): 1, ('always', 'love'): 1, ('love', 'you'): 1, ('you', 'that'): 1, ('that', 'much'): 1})
clipped_counts: {('know', 'that'): 0, ('that', 'I'): 0, ('I', 'always'

0.38260294162784475

In [19]:
sentence_bleu([r1, r2], h2)

[['you', 'know', 'I', 'always', 'love', 'you', 'so', 'much'], ['know', 'right', 'I', 'always', 'love', 'you', 'a', 'lot']] [['you', 'love', 'I', 'always']]
At 1-gram
max_counts: {('you',): 2, ('love',): 1, ('I',): 1, ('always',): 1}
counts: Counter({('you',): 1, ('love',): 1, ('I',): 1, ('always',): 1})
clipped_counts: {('you',): 1, ('love',): 1, ('I',): 1, ('always',): 1}
4
p_1 -> numerator=4 denominator=4 by modified_precision with 1-Gram
At 2-gram
max_counts: {('you', 'love'): 0, ('love', 'I'): 0, ('I', 'always'): 1}
counts: Counter({('you', 'love'): 1, ('love', 'I'): 1, ('I', 'always'): 1})
clipped_counts: {('you', 'love'): 0, ('love', 'I'): 0, ('I', 'always'): 1}
3
p_2 -> numerator=1 denominator=3 by modified_precision with 2-Gram
At 3-gram
max_counts: {('you', 'love', 'I'): 0, ('love', 'I', 'always'): 0}
counts: Counter({('you', 'love', 'I'): 1, ('love', 'I', 'always'): 1})
clipped_counts: {('you', 'love', 'I'): 0, ('love', 'I', 'always'): 0}
2
p_3 -> numerator=0 denominator=2 by

C:\Users\ICTUSER\AppData\Local\Temp\ipykernel_7400\1682077881.py:25: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ICTUSER\AppData\Local\Temp\ipykernel_7400\1682077881.py:25: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


2.795279274196275e-05

In [20]:
sentence_bleu([r1, r2], h3)

[['you', 'know', 'I', 'always', 'love', 'you', 'so', 'much'], ['know', 'right', 'I', 'always', 'love', 'you', 'a', 'lot']] [['I', 'I', 'I', 'I', 'I']]
At 1-gram
max_counts: {('I',): 1}
counts: Counter({('I',): 5})
clipped_counts: {('I',): 1}
5
p_1 -> numerator=1 denominator=5 by modified_precision with 1-Gram
At 2-gram
max_counts: {('I', 'I'): 0}
counts: Counter({('I', 'I'): 4})
clipped_counts: {('I', 'I'): 0}
4
p_2 -> numerator=0 denominator=4 by modified_precision with 2-Gram
At 3-gram
max_counts: {('I', 'I', 'I'): 0}
counts: Counter({('I', 'I', 'I'): 3})
clipped_counts: {('I', 'I', 'I'): 0}
3
p_3 -> numerator=0 denominator=3 by modified_precision with 3-Gram
At 4-gram
max_counts: {('I', 'I', 'I', 'I'): 0}
counts: Counter({('I', 'I', 'I', 'I'): 2})
clipped_counts: {('I', 'I', 'I', 'I'): 0}
2
p_4 -> numerator=0 denominator=2 by modified_precision with 4-Gram
brevity penalty: 0.5488116360940264
p_numerator: Counter({1: 1, 2: 0, 3: 0, 4: 0})
p_denominators: Counter({1: 5, 2: 4, 3: 3, 4:

C:\Users\ICTUSER\AppData\Local\Temp\ipykernel_7400\1682077881.py:25: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ICTUSER\AppData\Local\Temp\ipykernel_7400\1682077881.py:25: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ICTUSER\AppData\Local\Temp\ipykernel_7400\1682077881.py:25: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


3.670124608961276e-07

In [21]:
0.25*6/8 + 0.25*3/7 + 0.25*2/6 + 0.25*1/5

0.42797619047619045

In [22]:
0.25*4/4 + 0.25*1/3 + 0.25*0/2 + 0.25*0/1

0.3333333333333333

In [23]:
0.25*1/5 + 0.25*0/4 + 0.25*0/3 + 0.25*0/2

0.05

In [24]:
log = math.log

In [25]:
0.25*log(6/8) + 0.25*log(3/7) + 0.25*log(2/6) + 0.25*log(1/5)

-0.9607575334852987

In [26]:
0.25*log(4/4) + 0.25*log(1/3) + 0.25*log(1/100000000) + 0.25*log(1/100000000)

-9.48499344414321

In [27]:
0.25*log(1/5) + 0.25*log(1/100000000) + 0.25*log(1/100000000) + 0.25*log(1/100000000)

-14.217870036072801

In [28]:
exp = math.exp

In [29]:
exp(-0.9607575334852987)

0.38260294162784475

In [30]:
exp(-9.48499344414321)

7.598356856515924e-05

In [31]:
exp(-14.217870036072801)

6.687403049764207e-07

In [32]:
r1

['you', 'know', 'I', 'always', 'love', 'you', 'so', 'much']

In [33]:
r2

['know', 'right', 'I', 'always', 'love', 'you', 'a', 'lot']

In [36]:
0.36787944117144233 * 7.598356856515924e-05

2.795279274196275e-05

In [38]:
0.5488116360940264 * 6.687403049764207e-07

3.670124608961276e-07